In [1]:
import copy
import itertools
import os
import time

import matplotlib.pyplot as plt
import numpy as np
import pywt
from IPython.display import clear_output
from pynufft import NUFFT
from scipy import signal as sci_signal
from scipy.constants import c

from csromer.base import Dataset
from csromer.dictionaries.discrete import DiscreteWavelet
from csromer.dictionaries.undecimated import UndecimatedWavelet
from csromer.io import Reader, Writer
from csromer.objectivefunction import L1, TSV, TV, Chi2, OFunction
from csromer.optimization import ADMM, FISTA, SDMM, GradientBasedMethod
from csromer.reconstruction import Parameter
from csromer.simulation import FaradayThickSource, FaradayThinSource
from csromer.transformers import DFT1D, NUFFT1D, Gridding
from csromer.utils import Gaussian, complex_to_real, real_to_complex

%matplotlib inline

#np.random.seed(666)

In [2]:
def gini_coefficient(w):
    # Order vector
    w_ordered = np.sort(np.abs(w), kind="stable")
    l1_norm = np.sum(np.abs(w_ordered))
    M = len(w_ordered)
    m = np.arange(0, M)
    const = (M - m + 1.5) / M
    if l1_norm == 0.0:
        coeff = np.nan
    else:
        coeff = np.sum((w_ordered / l1_norm) * const)
    return 1.0 - 2.0 * coeff

In [3]:
def chi2_calc(residuals):
    if residuals.dtype == np.complex64 or residuals.dtype == np.complex128:
        data = residuals.real**2 + residuals.imag**2
    else:
        data = residuals**2
    return np.sum(data)

In [4]:
def aicbic(residuals, x):
    rss = chi2_calc(residuals)
    if x.dtype == np.complex64 or x.dtype == np.complex128:
        df = np.count_nonzero(x.real) + np.count_nonzero(x.imag)
    else:
        df = np.count_nonzero(x)
    l = 2 * len(residuals)
    return l * np.log(rss / l) + 2 * df, l * np.log(rss / l) + df * np.log(l)

In [5]:
def list_to2darray(x: list = None, cols: int = None, dtype=None):
    b = list(map(list, zip(*[iter(x)] * cols)))
    #b = [x[cols*i : cols*(i+1)] for i in range(rows)]
    if dtype is None:
        return np.array(b)
    else:
        return np.array(b, dtype=dtype)

In [6]:
class statistics:

    def __init__(self, m, n, z):
        self.sum = np.zeros((m, n), dtype=np.float32)
        self.sum2 = np.zeros((m, n), dtype=np.float32)
        self.n = z * np.ones((m, n), dtype=np.int32)

    def cumul(self, x):
        x_values = np.where(x != np.nan, x, 0.0)
        subtract = np.where(x == np.nan, -1, 0)
        self.sum += x_values
        self.sum2 += x_values * x_values
        self.n += subtract

    def mean(self):
        return np.where(self.n > 0, self.sum / self.n, np.nan)

    def std(self):
        return np.where(
            self.n > 0, np.sqrt(self.sum2 / self.n - self.sum * self.sum / self.n / self.n), np.nan
        )

In [7]:
class Test:

    def __init__(
        self,
        nu_min=None,
        nu_max=None,
        nchannels=None,
        noise_frac=None,
        remove_frac=None,
        use_gridding=False,
        ftransform="nufft",
        use_wavelet=None,
        source_1=None,
        source_2=None,
        scenario=1,
        append_signal=False
    ):
        self.nu_min = nu_min
        self.nu_max = nu_max
        self.nchannels = nchannels
        self.noise_frac = noise_frac
        self.remove_frac = remove_frac
        self.use_gridding = use_gridding
        self.use_wavelet = use_wavelet
        self.ftransform = ftransform
        self.scenario = scenario
        self.append_signal = append_signal
        self.nu = np.linspace(start=nu_min, stop=nu_max, num=nchannels)
        self.source_1 = copy.deepcopy(source_1)
        self.source_2 = copy.deepcopy(source_2)

        if self.source_1 is not None:
            self.source_1.nu = self.nu
            self.source_1.l2_ref = self.source_1.calculate_l2ref()
            self.source_1.simulate()

        if self.source_2 is not None:
            self.source_2.nu = self.nu
            self.source_2.l2_ref = self.source_2.calculate_l2ref()
            self.source_2.simulate()

        if scenario == 1:
            self.source = self.source_1
        elif scenario == 2:
            self.source = self.source_2
        elif scenario == 3:
            self.source = self.source_1 + self.source_2
        else:
            raise ValueError("This scenario does not exist")

        if remove_frac:
            self.source.remove_channels(remove_frac, np.random.RandomState(int(time.time())))

        self.noiseless_source = copy.deepcopy(self.source)

        if scenario == 1:
            self.avg_signal = np.abs(self.source_1.s_nu)
        elif scenario == 2:
            self.avg_signal = np.abs(self.source_2.s_nu)
        else:
            self.avg_signal = (np.abs(self.source_1.s_nu) + np.abs(self.source_2.s_nu)) / 2.0

        if noise_frac:
            self.source.apply_noise(self.avg_signal * self.noise_frac)

        if use_gridding:
            gridding = Gridding(self.source)
            gridding_noiseless = Gridding(self.noiseless_source)
            self.source = gridding.run()
            self.noiseless_source = gridding_noiseless.run()

    def apply_noise(self):
        if self.noise_frac:
            self.source.apply_noise(
                self.avg_signal * self.noise_frac, np.random.RandomState(int(time.time()))
            )

    def run(self, lambda_tv: float = None, lambda_tsv: float = None):
        self.parameter = Parameter()
        self.parameter.calculate_cellsize(dataset=self.source)

        dft = DFT1D(dataset=self.source, parameter=self.parameter)

        self.F_dirty = dft.backward(self.source.data)

        if self.use_wavelet:
            self.wavelet = DiscreteWavelet(wavelet_name=self.use_wavelet, mode="periodization", append_signal=self.append_signal)
            #self.wavelet = UndecimatedWavelet(
            #    wavelet_name=self.use_wavelet,
            #    mode="periodization",
            #    append_signal=self.append_signal
            #)
            self.lambda_l1 = np.sqrt(len(self.source.data) + 2 * np.sqrt(len(self.source.data))
                                     ) * 2.0 * np.sqrt(2) * np.mean(self.source.sigma)
        else:
            self.lambda_l1 = np.sqrt(len(self.source.data) + 2 * np.sqrt(len(self.source.data))
                                     ) * np.sqrt(2) * np.mean(self.source.sigma)

        if lambda_tv is None:
            lambda_tv = 0.0

        if lambda_tsv is None:
            lambda_tsv = 0.0

        if self.ftransform == "nufft":
            nufft = NUFFT1D(dataset=self.source, parameter=self.parameter, solve=True)
            if self.use_wavelet:
                chi2 = Chi2(dft_obj=nufft, wavelet=self.wavelet)
            else:
                chi2 = Chi2(dft_obj=nufft)
        else:
            if self.use_wavelet:
                chi2 = Chi2(dft_obj=dft, wavelet=self.wavelet)
            else:
                chi2 = Chi2(dft_obj=dft)

        l1 = L1(reg=self.lambda_l1)
        tsv = TSV(reg=lambda_tsv)
        tv = TV(reg=lambda_tv)
        F_func = [chi2, l1, tsv]
        f_func = [chi2]
        g_func = [l1, tsv]

        F_obj = OFunction(F_func)
        f_obj = OFunction(f_func)
        g_obj = OFunction(g_func)

        self.parameter.data = self.F_dirty

        self.parameter.complex_data_to_real()

        if self.use_wavelet:
            self.parameter.data = self.wavelet.decompose(self.parameter.data)
            opt = FISTA(
                guess_param=self.parameter,
                F_obj=F_obj,
                fx=chi2,
                gx=g_obj,
                noise=2.0 * self.source.theo_noise,
                verbose=False
            )
        else:
            opt = FISTA(
                guess_param=self.parameter,
                F_obj=F_obj,
                fx=chi2,
                gx=g_obj,
                noise=self.source.theo_noise,
                verbose=False
            )

        self.obj, self.X = opt.run()

        if self.use_wavelet is not None:
            self.coeffs = copy.deepcopy(self.X.data)
            k = np.count_nonzero(self.coeffs)
            self.sparsity = k / len(self.coeffs)
            self.gini = gini_coefficient(self.coeffs)
            self.X.data = self.wavelet.reconstruct(self.X.data)
            self.aic, self.bic = aicbic(self.source.residual, self.coeffs)

        else:
            k = np.count_nonzero(self.X.data)
            self.sparsity = k / len(self.X.data)
            self.gini = gini_coefficient(self.X.data)
            self.aic, self.bic = aicbic(self.source.residual, self.X.data)

        self.X.real_data_to_complex()

        self.X_residual = dft.backward(self.source.residual)

        self.X_restored = self.X.convolve() + self.X_residual

        self.res_noise = 0.5 * (np.std(self.X_residual.real) + np.std(self.X_residual.imag))
        self.rmse = np.sqrt(
            np.sum(self.source.residual.real**2 + self.source.residual.imag**2) /
            (2 * len(self.source.residual))
        )
        meaningful_signal = np.where(np.abs(self.parameter.phi) < self.parameter.max_faraday_depth)
        self.signal = np.mean(np.abs(self.X_restored[meaningful_signal]))
        self.peak_signal = np.max(np.abs(self.X_restored))

        self.snr = self.signal / self.res_noise
        self.psnr = self.peak_signal / self.res_noise

        print("Signal-to-noise ratio: {0}".format(self.snr))
        print("Peak Signal-to-noise ratio: {0}".format(self.psnr))
        print("Standard deviation: ({0})*10**5".format(self.res_noise * 10**5))
        print("Root Mean Squared Error: ({0})*10**5".format(self.rmse * 10**5))

        # self.lags, self.autocorr_res, self.autocorr_res_sq, self.bound, self.percentage_real_in, self.percentage_imag_in, self.percentage_real_in_sq, self.percentage_imag_in_sq = self.source.assess_residuals()

        # self.residual_comparison = self.noiseless_source.data - self.source.model_data
        """
        
        self.fig, self.ax = plt.subplots(nrows=2, ncols=4, sharey='row', figsize=(18, 5))

        # Data
        self.ax[0,0].plot(self.source.lambda2, self.source.data.real, 'k.', label=r"Stokes $Q$")
        self.ax[0,0].plot(self.source.lambda2, self.source.data.imag, 'c.', label=r"Stokes $U$")
        self.ax[0,0].plot(self.source.lambda2, np.abs(self.source.data), 'g.', label=r"$|P|$")
        self.ax[0,0].set_xlabel(r'$\lambda^2$[m$^{2}$]')
        self.ax[0,0].set_ylabel(r'Jy/beam')
        self.ax[0,0].title.set_text("Data")

        self.ax[1,0].plot(self.parameter.phi, self.F_dirty.real, 'c--', label=r"Stokes $Q$")
        self.ax[1,0].plot(self.parameter.phi, self.F_dirty.imag, 'c:', label=r"Stokes $U$")
        self.ax[1,0].plot(self.parameter.phi, np.abs(self.F_dirty), 'k-', label=r"|P|")
        self.ax[1,0].set_xlabel(r'$\phi$[rad m$^{-2}$]')
        self.ax[1,0].set_ylabel(r'Jy/beam m$^2$ rad$^{-1}$ rmtf$^{-1}$')
        self.ax[1,0].set_xlim([-1000,1000])

        # Model
        self.ax[0,1].plot(self.source.lambda2, self.source.model_data.real, 'k.', label=r"Stokes $Q$")
        self.ax[0,1].plot(self.source.lambda2, self.source.model_data.imag, 'c.', label=r"Stokes $U$")
        self.ax[0,1].plot(self.source.lambda2, np.abs(self.source.model_data), 'g.', label=r"$|P|$")
        self.ax[0,1].set_xlabel(r'$\lambda^2$[m$^{2}$]')
        self.ax[0,1].set_ylabel(r'Jy/beam')
        self.ax[0,1].title.set_text("Model")

        self.ax[1,1].get_shared_y_axes().remove(self.ax[1,1])
        self.ax[1,1].clear()
        self.ax[1,1].plot(self.parameter.phi, self.X.data.real, 'c--', label=r"Stokes $Q$")
        self.ax[1,1].plot(self.parameter.phi, self.X.data.imag, 'c:', label=r"Stokes $U$")
        self.ax[1,1].plot(self.parameter.phi, np.abs(self.X.data), 'k-', label=r"$|P|$")
        self.ax[1,1].set_xlabel(r'$\phi$[rad m$^{-2}$]')
        self.ax[1,1].set_ylabel(r'Jy/beam m$^2$ rad$^{-1}$ pix$^{-1}$')
        self.ax[1,1].set_xlim([-1000,1000])
        
        # Residual

        self.ax[0,2].plot(self.source.lambda2, self.source.residual.real, 'k.', label=r"Stokes $Q$")
        self.ax[0,2].plot(self.source.lambda2, self.source.residual.imag, 'c.', label=r"Stokes $U$")
        self.ax[0,2].plot(self.source.lambda2, np.abs(self.source.residual), 'g.', label=r"$|P|$")
        self.ax[0,2].set_xlabel(r'$\lambda^2$[m$^{2}$]')
        self.ax[0,2].set_ylabel(r'Jy/beam')
        self.ax[0,2].title.set_text("Residual")

        self.ax[1,2].plot(self.parameter.phi, self.X_residual.real, 'c--', label=r"Stokes $Q$")
        self.ax[1,2].plot(self.parameter.phi, self.X_residual.imag, 'c:', label=r"Stokes $U$")
        self.ax[1,2].plot(self.parameter.phi, np.abs(self.X_residual), 'k-', label=r"$|P|$")
        self.ax[1,2].set_xlabel(r'$\phi$[rad m$^{-2}$]')
        self.ax[1,2].set_ylabel(r'Jy/beam m$^2$ rad$^{-1}$ rmtf$^{-1}$')
        self.ax[1,2].set_xlim([-1000,1000])
        
        if self.use_wavelet:
            self.ax[0,3].get_shared_y_axes().remove(self.ax[0,3])
            self.ax[0,3].clear()
            self.ax[0,3].plot(self.coeffs)
            self.ax[0,3].title.set_text("Coefficients")

        self.ax[1,3].plot(self.parameter.phi, self.X_restored.real, 'c--', label=r"Stokes $Q$")
        self.ax[1,3].plot(self.parameter.phi, self.X_restored.imag, 'c:', label=r"Stokes $U$")
        self.ax[1,3].plot(self.parameter.phi, np.abs(self.X_restored), 'k-', label=r"$|P|$")
        self.ax[1,3].set_xlim([-1000,1000])
        self.ax[1,3].set_xlabel(r'$\phi$[rad m$^{-2}$]')
        self.ax[1,3].set_ylabel(r'Jy/beam m$^2$ rad$^{-1}$ rmtf$^{-1}$')
        self.ax[1,3].title.set_text("Restored")
        
        self.fig.tight_layout()
        """

In [8]:
def run_test(
    source_1,
    source_2,
    nsigma,
    remove_frac,
    nu_min=1.008e9,
    nu_max=2.031e9,
    nchannels=1000,
    scenario=1,
    use_wavelet=None,
    append_signal=False
):
    nosigma_objs = []
    for remv_frac in remove_frac:
        nosigma_objs.append(
            Test(
                nu_min=nu_min,
                nu_max=nu_max,
                nchannels=nchannels,
                noise_frac=0.0,
                remove_frac=remv_frac,
                scenario=scenario,
                source_1=source_1,
                source_2=source_2,
                use_wavelet=use_wavelet,
                append_signal=append_signal
            )
        )

    test_objs = []
    for nsig in nsigma:
        for i in range(0, len(remove_frac)):
            copy_object = copy.deepcopy(nosigma_objs[i])
            copy_object.noise_frac = nsig
            copy_object.apply_noise()
            test_objs.append(copy_object)

    del nosigma_objs

    nid = len(nsigma) * len(remove_frac)
    for _id in range(0, nid):
        test_objs[_id].run()

    return test_objs

In [9]:
# JVLA 1.008 - 2.031 GHz 546 channels
# MeerKAT 0.9 GHz-1.420 GHz 546
# eMERLIN 1.230 - 1.740 GHz 4096
def run_tests(
    source_1,
    source_2,
    nsigma,
    remove_frac,
    nsamples,
    nu_min=1.008e9,
    nu_max=2.031e9,
    nchannels=1000,
    scenario=1,
    use_wavelet=None,
    append_signal=False
):
    m = len(nsigma)
    n = len(remove_frac)
    psnrs = statistics(m, n, nsamples)
    rmses = statistics(m, n, nsamples)
    #noises = statistics(m, n, nsamples)
    sparsities = statistics(m, n, nsamples)
    #ginies = statistics(m, n, nsamples)
    aics = statistics(m, n, nsamples)
    bics = statistics(m, n, nsamples)

    for i in range(0, nsamples):
        test = run_test(
            source_1,
            source_2,
            nsigma,
            remove_frac,
            nu_min=nu_min,
            nu_max=nu_max,
            nchannels=nchannels,
            scenario=scenario,
            use_wavelet=use_wavelet,
            append_signal=append_signal
        )
        psnrs.cumul(list_to2darray([x.psnr for x in test], n, dtype=np.float32))
        #noises.cumul(list_to2darray([x.res_noise for x in test], n, dtype=np.float32))
        sparsities.cumul(list_to2darray([x.sparsity * 100.0 for x in test], n, dtype=np.float32))
        #ginies.cumul(list_to2darray([x.gini for x in test], n, dtype=np.float32))
        rmses.cumul(list_to2darray([x.rmse for x in test], n, dtype=np.float32))
        aics.cumul(list_to2darray([x.aic for x in test], n, dtype=np.float32))
        bics.cumul(list_to2darray([x.bic for x in test], n, dtype=np.float32))
        for t in test:
            del t
        test = []

    psnr_mean, psnr_std = psnrs.mean(), psnrs.std()
    rmse_mean, rmse_std = rmses.mean(), rmses.std()
    #noise_mean, noise_std = noises.mean(), noises.std()
    sparsity_mean, sparsity_std = sparsities.mean(), sparsities.std()
    #gini_mean, gini_std = ginies.mean(), ginies.std()
    aic_mean, aic_std = aics.mean(), aics.std()
    bic_mean, bic_std = bics.mean(), bics.std()
    return psnr_mean, psnr_std, rmse_mean, rmse_std, aic_mean, aic_std, bic_mean, bic_std, sparsity_mean, sparsity_std

In [10]:
#source_1 = FaradayThinSource(s_nu=0.0035, phi_gal=-200, spectral_idx=1.0)
source_1 = FaradayThinSource(s_nu=0.0035, phi_gal=-400, spectral_idx=0.7)
#source_2 = FaradayThickSource(s_nu=0.0035, phi_fg=140, phi_center=200, spectral_idx=1.0)
source_2 = FaradayThickSource(s_nu=0.0035, phi_fg=95, phi_center=200, spectral_idx=0.7)

In [11]:
nsigma = [0.2]
remove_frac = [0.3]
scenarios = [1, 2, 3]
family_per_scenario = ["coif", "sym", "dmey"]
wavelist_per_scenario = [pywt.wavelist(i, kind="discrete") for i in family_per_scenario]

In [12]:
names = ["PSNR", "RMSE", "AIC", "BIC"]
samples = 50
scenario_1_means = np.empty((len(names), len(wavelist_per_scenario[0])), dtype=np.float32)
scenario_1_stds = np.empty((len(names), len(wavelist_per_scenario[0])), dtype=np.float32)

scenario_2_means = np.empty((len(names), len(wavelist_per_scenario[1])), dtype=np.float32)
scenario_2_stds = np.empty((len(names), len(wavelist_per_scenario[1])), dtype=np.float32)

scenario_3_means = np.empty((len(names), len(wavelist_per_scenario[2])), dtype=np.float32)
scenario_3_stds = np.empty((len(names), len(wavelist_per_scenario[2])), dtype=np.float32)

append_signal = False

for i in range(len(scenarios)):
    for j in range(len(wavelist_per_scenario[i])):
        psnr_mean, psnr_std, rmse_mean, rmse_std, aic_mean, aic_std, bic_mean, bic_std, sparsity_mean, sparsity_std = run_tests(
            source_1,
            source_2,
            nsigma,
            remove_frac,
            samples,
            scenario=scenarios[i],
            use_wavelet=wavelist_per_scenario[i][j],
            append_signal=append_signal,
            nu_min=0.9e9,
            nu_max=1.67e9
        )
        if i == 0:
            scenario_1_means[0, j] = psnr_mean
            scenario_1_means[1, j] = rmse_mean
            scenario_1_means[2, j] = aic_mean
            scenario_1_means[3, j] = bic_mean

            scenario_1_stds[0, j] = psnr_std
            scenario_1_stds[1, j] = rmse_std
            scenario_1_stds[2, j] = aic_std
            scenario_1_stds[3, j] = bic_std
        elif i == 1:
            scenario_2_means[0, j] = psnr_mean
            scenario_2_means[1, j] = rmse_mean
            scenario_2_means[2, j] = aic_mean
            scenario_2_means[3, j] = bic_mean

            scenario_2_stds[0, j] = psnr_std
            scenario_2_stds[1, j] = rmse_std
            scenario_2_stds[2, j] = aic_std
            scenario_2_stds[3, j] = bic_std
        else:
            scenario_3_means[0, j] = psnr_mean
            scenario_3_means[1, j] = rmse_mean
            scenario_3_means[2, j] = aic_mean
            scenario_3_means[3, j] = bic_mean

            scenario_3_stds[0, j] = psnr_std
            scenario_3_stds[1, j] = rmse_std
            scenario_3_stds[2, j] = aic_std
            scenario_3_stds[3, j] = bic_std

FWHM of the main peak of the RMTF: 46.990 rad/m^2
Maximum recovered width structure: 92.687 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 16423.131
Signal-to-noise ratio: 1.393763887438697
Peak Signal-to-noise ratio: 119.49213900777009
Standard deviation: (2.7832466003019363)*10**5
Root Mean Squared Error: (73.10813071420237)*10**5
FWHM of the main peak of the RMTF: 44.122 rad/m^2
Maximum recovered width structure: 97.396 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 15420.676
Signal-to-noise ratio: 1.4328745363342847
Peak Signal-to-noise ratio: 112.82970951019796
Standard deviation: (2.9658287530764937)*10**5
Root Mean Squared Error: (80.74674942490087)*10**5
FWHM of the main peak of the RMTF: 44.105 rad/m^2
Maximum recovered width structure: 97.486 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 15414.831
Signal-to-noise ratio: 1.404888578519903
Peak Signal-to-noise ratio: 107.97275818390364
Standard de

Signal-to-noise ratio: 1.4160403891906743
Peak Signal-to-noise ratio: 120.73377522500724
Standard deviation: (2.752398359007202)*10**5
Root Mean Squared Error: (76.5283646808274)*10**5
FWHM of the main peak of the RMTF: 47.803 rad/m^2
Maximum recovered width structure: 94.185 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 16707.233
Signal-to-noise ratio: 1.418689561362875
Peak Signal-to-noise ratio: 117.748665061042
Standard deviation: (2.898061393352691)*10**5
Root Mean Squared Error: (76.66179018147703)*10**5
FWHM of the main peak of the RMTF: 44.105 rad/m^2
Maximum recovered width structure: 97.486 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 15414.831
Signal-to-noise ratio: 1.3608566166153575
Peak Signal-to-noise ratio: 112.48812757628505
Standard deviation: (2.9756303774775006)*10**5
Root Mean Squared Error: (81.31095440154323)*10**5
FWHM of the main peak of the RMTF: 44.105 rad/m^2
Maximum recovered width structure: 97.486 r

Signal-to-noise ratio: 1.4101286501784118
Peak Signal-to-noise ratio: 103.90043456704282
Standard deviation: (2.938563011412043)*10**5
Root Mean Squared Error: (78.86523603343882)*10**5
FWHM of the main peak of the RMTF: 46.302 rad/m^2
Maximum recovered width structure: 97.396 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 16182.503
Signal-to-noise ratio: 1.5036116347311133
Peak Signal-to-noise ratio: 109.76190539023635
Standard deviation: (2.9243517928989604)*10**5
Root Mean Squared Error: (78.34686144459265)*10**5
FWHM of the main peak of the RMTF: 44.105 rad/m^2
Maximum recovered width structure: 97.486 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 15414.831
Signal-to-noise ratio: 1.4640890134578086
Peak Signal-to-noise ratio: 113.97945749358854
Standard deviation: (2.875575955840759)*10**5
Root Mean Squared Error: (81.60835736904049)*10**5
FWHM of the main peak of the RMTF: 44.105 rad/m^2
Maximum recovered width structure: 97.4

Signal-to-noise ratio: 1.4416853854840368
Peak Signal-to-noise ratio: 125.56773403805063
Standard deviation: (2.7040492568630725)*10**5
Root Mean Squared Error: (72.33917676888773)*10**5
FWHM of the main peak of the RMTF: 48.341 rad/m^2
Maximum recovered width structure: 96.409 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 16895.085
Signal-to-noise ratio: 1.4680587414639694
Peak Signal-to-noise ratio: 123.83122589046978
Standard deviation: (2.6844812964554876)*10**5
Root Mean Squared Error: (71.2986106222759)*10**5
FWHM of the main peak of the RMTF: 44.105 rad/m^2
Maximum recovered width structure: 97.486 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 15414.831
Signal-to-noise ratio: 1.4221209800226133
Peak Signal-to-noise ratio: 124.20459347384697
Standard deviation: (2.72143843176309)*10**5
Root Mean Squared Error: (71.81799708576489)*10**5
FWHM of the main peak of the RMTF: 45.543 rad/m^2
Maximum recovered width structure: 96.76

Signal-to-noise ratio: 1.425235839830819
Peak Signal-to-noise ratio: 118.90015273188912
Standard deviation: (2.762306758086197)*10**5
Root Mean Squared Error: (74.24311523395875)*10**5
FWHM of the main peak of the RMTF: 46.017 rad/m^2
Maximum recovered width structure: 88.523 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 16082.950
Signal-to-noise ratio: 1.4282089287742379
Peak Signal-to-noise ratio: 116.06312541246497
Standard deviation: (2.7595075152930804)*10**5
Root Mean Squared Error: (75.4250275673379)*10**5
FWHM of the main peak of the RMTF: 45.157 rad/m^2
Maximum recovered width structure: 92.249 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 15782.453
Signal-to-noise ratio: 1.4229344668437724
Peak Signal-to-noise ratio: 105.87749776962478
Standard deviation: (2.8625390768866055)*10**5
Root Mean Squared Error: (81.58868413398875)*10**5
FWHM of the main peak of the RMTF: 44.105 rad/m^2
Maximum recovered width structure: 97.48

...

Signal-to-noise ratio: 1.5260179049912992
Peak Signal-to-noise ratio: 60.56655352004677
Standard deviation: (2.7491205401020125)*10**5
Root Mean Squared Error: (71.5121758018251)*10**5
FWHM of the main peak of the RMTF: 45.730 rad/m^2
Maximum recovered width structure: 97.486 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 15982.495
Signal-to-noise ratio: 1.4904599859512715
Peak Signal-to-noise ratio: 55.459641766371746
Standard deviation: (2.771414438029751)*10**5
Root Mean Squared Error: (75.42672515239083)*10**5
FWHM of the main peak of the RMTF: 45.968 rad/m^2
Maximum recovered width structure: 94.716 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 16065.826
Signal-to-noise ratio: 1.5063345542121374
Peak Signal-to-noise ratio: 57.53665825562232
Standard deviation: (2.925279113696888)*10**5
Root Mean Squared Error: (76.26146637312947)*10**5
FWHM of the main peak of the RMTF: 44.482 rad/m^2
Maximum recovered width structure: 95.516 

Signal-to-noise ratio: 1.404768617353437
Peak Signal-to-noise ratio: 64.6737460028866
Standard deviation: (2.6849880669033155)*10**5
Root Mean Squared Error: (73.13813813141489)*10**5
FWHM of the main peak of the RMTF: 44.122 rad/m^2
Maximum recovered width structure: 97.396 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 15420.676
Signal-to-noise ratio: 1.4921294990100347
Peak Signal-to-noise ratio: 53.61475637235934
Standard deviation: (2.797493289108388)*10**5
Root Mean Squared Error: (75.16963630882658)*10**5
FWHM of the main peak of the RMTF: 44.105 rad/m^2
Maximum recovered width structure: 97.486 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 15414.831
Signal-to-noise ratio: 1.5423716044020168
Peak Signal-to-noise ratio: 47.178318693595365
Standard deviation: (2.926038359873928)*10**5
Root Mean Squared Error: (78.07018534580226)*10**5
FWHM of the main peak of the RMTF: 48.553 rad/m^2
Maximum recovered width structure: 87.497 r

Signal-to-noise ratio: 1.559681811534276
Peak Signal-to-noise ratio: 56.14856060172237
Standard deviation: (2.672609844012186)*10**5
Root Mean Squared Error: (75.31372454185289)*10**5
FWHM of the main peak of the RMTF: 44.105 rad/m^2
Maximum recovered width structure: 97.486 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 15414.831
Signal-to-noise ratio: 1.491627643256086
Peak Signal-to-noise ratio: 60.20663464522026
Standard deviation: (2.7866304662893526)*10**5
Root Mean Squared Error: (75.05741020987757)*10**5
FWHM of the main peak of the RMTF: 46.273 rad/m^2
Maximum recovered width structure: 87.497 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 16172.354
Signal-to-noise ratio: 1.406428578216193
Peak Signal-to-noise ratio: 64.34842876369662
Standard deviation: (2.765884346445091)*10**5
Root Mean Squared Error: (73.89937338939774)*10**5
FWHM of the main peak of the RMTF: 44.172 rad/m^2
Maximum recovered width structure: 97.126 rad

Signal-to-noise ratio: 1.617597345870707
Peak Signal-to-noise ratio: 102.5379930117319
Standard deviation: (2.9358581741689704)*10**5
Root Mean Squared Error: (80.21640056560561)*10**5
FWHM of the main peak of the RMTF: 44.212 rad/m^2
Maximum recovered width structure: 97.486 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 15452.078
Signal-to-noise ratio: 1.8285096428524643
Peak Signal-to-noise ratio: 94.28669431171663
Standard deviation: (2.8322378057055175)*10**5
Root Mean Squared Error: (79.34488506081372)*10**5
FWHM of the main peak of the RMTF: 44.257 rad/m^2
Maximum recovered width structure: 96.678 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 15467.887
Signal-to-noise ratio: 1.7102922537406524
Peak Signal-to-noise ratio: 111.77396307706073
Standard deviation: (2.796823355311062)*10**5
Root Mean Squared Error: (76.51152799408491)*10**5
FWHM of the main peak of the RMTF: 44.105 rad/m^2
Maximum recovered width structure: 97.486

Signal-to-noise ratio: 1.5605112941920358
Peak Signal-to-noise ratio: 99.13480815509652
Standard deviation: (3.0744562536710873)*10**5
Root Mean Squared Error: (89.00562781963049)*10**5
FWHM of the main peak of the RMTF: 44.534 rad/m^2
Maximum recovered width structure: 95.249 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 15564.800
Signal-to-noise ratio: 1.5569026480343267
Peak Signal-to-noise ratio: 102.82647371093432
Standard deviation: (2.8581745937117375)*10**5
Root Mean Squared Error: (81.29112290321062)*10**5
FWHM of the main peak of the RMTF: 44.951 rad/m^2
Maximum recovered width structure: 93.214 rad/m^2
Maximum Faraday Depth to which one has more than 50% sensitivity: 15710.220
Signal-to-noise ratio: 1.8426323913852924
Peak Signal-to-noise ratio: 90.29367199653849
Standard deviation: (3.037494025193155)*10**5
Root Mean Squared Error: (80.85197144351717)*10**5
FWHM of the main peak of the RMTF: 44.570 rad/m^2
Maximum recovered width structure: 95.07

In [13]:
undecimated = ""
radiotelescope = "meerkat"
np.save(undecimated + "wavelet_" + radiotelescope + "_scenario_1_means.npy", scenario_1_means)
np.save(undecimated + "wavelet_" + radiotelescope + "_scenario_1_stds.npy", scenario_1_stds)

np.save(undecimated + "wavelet_" + radiotelescope + "_scenario_2_means.npy", scenario_2_means)
np.save(undecimated + "wavelet_" + radiotelescope + "_scenario_2_stds.npy", scenario_2_stds)

np.save(undecimated + "wavelet_" + radiotelescope + "_scenario_3_means.npy", scenario_3_means)
np.save(undecimated + "wavelet_" + radiotelescope + "_scenario_3_stds.npy", scenario_3_stds)